In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

def carregar_dados(caminho_arquivo):
    """Carrega o dataset bruto do arquivo CSV."""
    return pd.read_csv(caminho_arquivo)

def realizar_feature_engineering(df):
    """
    Aplica as transformações de colunas e criação de novas variáveis.
    Inclui: One-Hot Encoding do arrival_mode e criação do alerta de oxigênio.
    """
    # One-Hot Encoding: Cria colunas como arrival_ambulance, arrival_walk_in, etc.
    df_processado = pd.get_dummies(df, columns=['arrival_mode'], prefix='arrival')
    
    # Criar Feature Auxiliar baseada em regra clínica
    df_processado['low_oxygen_alert'] = (df_processado['oxygen_saturation'] < 92).astype(int)
    
    return df_processado

def dividir_dados(df, target_col='triage_level'):
    """Separa os dados em atributos (X) e alvo (y) com divisão estratificada."""
    X = df.drop(target_col, axis=1)
    y = df[target_col]
    
    return train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

def escalonar_features(X_train, X_test, colunas_numericas):
    """Aplica a padronização (Z-score) nas colunas numéricas."""
    scaler = RobustScaler()
    
    # Garantindo que operamos em cópias para evitar avisos do Pandas
    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()
    
    X_train_scaled[colunas_numericas] = scaler.fit_transform(X_train[colunas_numericas])
    X_test_scaled[colunas_numericas] = scaler.transform(X_test[colunas_numericas])
    
    return X_train_scaled, X_test_scaled

def salvar_datasets(X_train, X_test, y_train, y_test, diretorio_saida="../data/processed/"):
    """Exporta os arquivos finais para a pasta de dados processados."""
    X_train.to_csv(f"{diretorio_saida}X_train.csv", index=False)
    X_test.to_csv(f"{diretorio_saida}X_test.csv", index=False)
    y_train.to_csv(f"{diretorio_saida}y_train.csv", index=False)
    y_test.to_csv(f"{diretorio_saida}y_test.csv", index=False)

def executar_pipeline_preprocessamento():
    """Função principal que coordena todas as etapas do pré-processamento."""
    # Definição de caminhos e colunas
    ARQUIVO_ENTRADA = "../data/raw/synthetic_medical_triage.csv"
    COLUNAS_NUMERICAS = [
        'age', 'heart_rate', 'systolic_blood_pressure', 
        'oxygen_saturation', 'body_temperature', 'pain_level', 
        'chronic_disease_count', 'previous_er_visits'
    ]

    # Passo 1: Carga
    df = carregar_dados(ARQUIVO_ENTRADA)

    # Passo 2: Engenharia de Features
    df = realizar_feature_engineering(df)

    # Passo 3: Divisão (Holdout)
    X_train, X_test, y_train, y_test = dividir_dados(df)

    # Passo 4: Escalonamento
    X_train, X_test = escalonar_features(X_train, X_test, COLUNAS_NUMERICAS)

    # Passo 5: Persistência
    salvar_datasets(X_train, X_test, y_train, y_test)
    
    print("Processamento concluído com sucesso e arquivos salvos!")


executar_pipeline_preprocessamento()

Processamento concluído com sucesso e arquivos salvos!
